In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import backtest
from plotting import plot_fills
from downloader import Downloader
from pure_funcs import denumpyize, numpyize, candidate_to_live_config, calc_spans, \
    analyze_fills, create_xk, round_dynamic, round_values
from njit_funcs import *
from procedures import dump_live_config, load_live_config, add_argparse_args, prepare_backtest_config, make_get_filepath
from time import time
import sys
import argparse
import pprint
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np

In [ ]:
plt.rcParams['figure.figsize'] = [29, 18]
pd.set_option('precision', 10)

In [ ]:
class Args:
    def __init__(self):
        self.backtest_config_path = 'configs/backtest/default.hjson'
        self.exchange = 'binance'
        self.symbol = 'XMRUSDT'
        self.market_type = 'futures'
        self.user = 'user_name'
        self.start_date = '2021-04-01'
        self.end_date = '2021-08-23'
        self.starting_balance = 1000.0
        self.starting_configs = ''
        self.base_dir = 'backtests'
config = await prepare_backtest_config(Args())
dl = Downloader(config)
sts = time()
data = await dl.get_sampled_ticks()
timestamps = data[:, 0]
qtys = data[:, 1]
prices = data[:, 2]
config['n_days'] = (timestamps[-1] - timestamps[0]) / (1000 * 60 * 60 * 24)

print(f'millis to load {len(prices)} ticks {(time() - sts) * 1000:.0f}ms')

In [ ]:
df = pd.DataFrame({'timestamp': timestamps, 'qty': qtys, 'price': prices}).set_index('timestamp')
df.price.iloc[::100].plot()

In [ ]:
# choose a slice on which to test
wsize_days = 240
ts = int(data[-1][0] - 60 * 60 * 24 * 1000 * wsize_days)
idx = np.argmax(data[:, 0] >= ts)
dataslice = data[idx:]

In [ ]:
hand_tuned = {
    "config_name": "hand_tuned",
    "logging_level": 0,
    "long": {"enabled": True,
             "grid_span": 0.17,
             "pbr_limit": 1.7,
             "max_n_entry_orders":  10,
             "initial_qty_pct":  0.01,
             "eprice_pprice_diff": 0.0025,
             "secondary_pbr_allocation": 0.5,
             "secondary_pprice_diff": 0.35,
             "eprice_exp_base": 1.618034,
             "min_markup": 0.0045,
             "markup_range":  0.0075,
             "n_close_orders": 7},
    "shrt": {"enabled": False,
             "grid_span": 0.1,
             "pbr_limit": 1.0,
             "max_n_entry_orders":  10,
             "initial_qty_pct":  0.01,
             "eprice_pprice_diff": 0.0025,
             "secondary_pbr_allocation": 0.5,
             "secondary_pprice_diff": 0.35,
             "eprice_exp_base": 1.618034,
             "min_markup": 0.0025,
             "markup_range":  0.0075,
             "n_close_orders": 7},
}

# round for better human readability
hand_tuned = round_values(hand_tuned, 4)
config['starting_balance'] = 1000.0
config_to_test = {**config, **numpyize(hand_tuned)}
dump_live_config(config_to_test, make_get_filepath(f'tmp/hand_tuned.json'))
hand_tuned

In [ ]:
sts = time()
fills, stats = backtest(config_to_test, dataslice)
elapsed = time() - sts
print(f'seconds elapsed {elapsed:.4f}')
fdf, sdf, analysis = analyze_fills(fills, stats, config_to_test)
pprint.pprint(analysis)
fdf

In [ ]:
sdf

In [ ]:
sdf.balance.plot()
sdf.equity.plot()

In [ ]:
plot_fills(df, fdf, plot_whole_df=True)

In [ ]:
fdf.psize.plot()

In [ ]:
period = 60 * 24 * 1 # daily
periodics = [sdf.balance.iloc[i + period] / sdf.balance.iloc[i] - 1 for i in range(0, len(sdf) - period, period)]
periodics = pd.Series(periodics)
periodics.plot()

In [ ]:
adgs = (sdf.balance / sdf.balance.iloc[0]) ** (1 / ((sdf.timestamp - sdf.timestamp.iloc[0]) / (1000 * 60 * 60 * 24)))
adgs.plot()

In [ ]:
sdf.price.plot()
sdf.long_pprice.replace(0.0, np.nan).plot()

In [ ]:
lpprices = sdf[sdf.long_pprice != 0.0]
pa_closeness_long = (lpprices.long_pprice - lpprices.price).abs() / lpprices.price
pa_closeness_long.plot()

In [ ]:
balance = config_to_test['starting_balance']
highest_bid = df.price.iloc[-1]

inverse = config_to_test['inverse']
do_long = True
qty_step = config_to_test['qty_step']
price_step = config_to_test['price_step']
min_qty = config_to_test['min_qty']
min_cost = config_to_test['min_cost']
c_mult = config_to_test['c_mult']

grid_span = config_to_test['long']['grid_span']
pbr_limit = config_to_test['long']['pbr_limit']
max_n_entry_orders = config_to_test['long']['max_n_entry_orders']
initial_qty_pct = config_to_test['long']['initial_qty_pct']
eprice_pprice_diff = config_to_test['long']['eprice_pprice_diff']
eprice_exp_base = config_to_test['long']['eprice_exp_base']
secondary_pbr_allocation = config_to_test['long']['secondary_pbr_allocation']
secondary_pprice_diff = config_to_test['long']['secondary_pprice_diff']

grid = calc_whole_long_entry_grid(
    balance, highest_bid, inverse, qty_step, price_step, min_qty, min_cost,
    c_mult, grid_span, pbr_limit, max_n_entry_orders, initial_qty_pct,
    eprice_pprice_diff, secondary_pbr_allocation, secondary_pprice_diff,
    eprice_exp_base=eprice_exp_base)
gdf = pd.DataFrame(grid, columns=['qty', 'price', 'psize', 'pprice', 'pbr'])
    
gdf.loc[:, 'eprice_pprice_diff'] = abs(gdf.price - gdf.pprice) / gdf.price
gdf.loc[:,'ddown_factor'] = gdf.qty / gdf.psize.shift()
gdf.loc[:,'bkr_price'] = gdf.apply(
    lambda x: calc_bankruptcy_price(balance, x.psize, x.pprice, 0.0, 0.0, inverse, c_mult), axis=1)
colors = 'rbygcmk'
for i, e in enumerate(gdf.itertuples()):
    plt.plot([0, 2], [e.price, e.price], f'{colors[i%len(colors)]}-')
    plt.plot([1, 2], [e.pprice, e.pprice], f'{colors[i%len(colors)]}--')
gdf